In [10]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import statistics
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [12]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
#! Set these variables
work_dir_name = "pineal1"

In [5]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"

Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [6]:
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="flair.t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels("pineal", suffix_list=["CH", "SRS"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-12-02 14:51:39.014 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/8 [00:00<?, ?it/s]2024-12-02 14:51:39.027 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-CH.nii.gz for Scan(subid=1010, sesid=20180208)


2024-12-02 14:51:39.769 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-SRS.nii.gz for Scan(subid=1280, sesid=20220317)
 25%|██▌       | 2/8 [00:00<00:02,  2.66it/s]2024-12-02 14:51:39.818 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-SRS.nii.gz for Scan(subid=1029, sesid=20170816)
2024-12-02 14:51:39.823 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-CH.nii.gz for Scan(subid=1188, sesid=20200720)
2024-12-02 14:51:39.863 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-SRS.nii.gz for Scan(subid=1272, sesid=20211105)
2024-12-02 14:51:39.892 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-SRS.nii.gz for Scan(subid=1321, sesid=20201020)
 75%|███████▌  | 6/8 [00:00<00:00,  8.31it/s]2024-12-02 14:51:39.909 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-SRS.nii.gz for Scan(subid=1265, sesid=20180127)
2024-

Calculate Dice Scores

In [16]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.label_path).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1010: 0.57
1029: 0.54
1188: 0.49
1265: 0.64
1272: 0.65
1280: 0.10
1321: 0.31
2187: 0.82
----------
Mean: 0.52
Std: 0.22
[0.5701754385964912, 0.5442477876106194, 0.4939271255060729, 0.6403785488958991, 0.6455026455026455, 0.09895833333333333, 0.3076923076923077, 0.8205128205128205]


In [17]:
iou = []
print("IoU")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pineal", ["CH", "SRS"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.iou(seg1, seg2, seg2_val=1)
    iou.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(iou) / len(iou)))
print("Std: {:0.2f}".format(statistics.stdev(iou)))

print(iou)

IoU
1010: 0.40
1029: 0.37
1188: 0.33
1265: 0.47
1272: 0.48
1280: 0.05
1321: 0.18
2187: 0.70
----------
Mean: 0.37
Std: 0.20
[0.3987730061349693, 0.3738601823708207, 0.3279569892473118, 0.4709976798143852, 0.4765625, 0.052054794520547946, 0.18181818181818182, 0.6956521739130435]


Calculate Hausdorff Dist for pineal

In [18]:
haus = []
print("haus")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pineal", ["CH", "SRS"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=1)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

haus
1010: 136.31
1029: 65.54
1188: 3.74
1265: 3.61
1272: 41.16
1280: 117.07
1321: 5.92
2187: 78.13
----------
Mean: 56.43
Std: 52.05
[136.31214179228496, 65.54387843269576, 3.7416573867739413, 3.605551275463989, 41.15823125451335, 117.06835610018618, 5.916079783099616, 78.12809993849844]


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [12]:
for inference in ensemble_out_dataset:
    label_path = inference.label_path
    new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
    utils.set_label_value(label_path, new_label_path, 2)

Produce commands to open images and labels in itksnap

In [15]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.root / new_label_path.name
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction, manual)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids

Produce commands to open images and labels in freeview

In [23]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat